In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, ShuffleSplit
import networkx as nx

import itertools

from pprint import pprint

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [2]:
import tensorflow as tf

In [3]:
from common.adjacency_list_to_graph import build_graph
from common.calculate_spring_rank import calculate_spring_rank
from common.graph_to_matrix import build_matrix
from tensorflow.python.client import device_lib

# Statistical significance

This page shows and explains the process of finding out SpringRank ranks statistical significance, that are extremely important to get really impressive results during genesis generation process (ahem, during the next few months).

# Table of contents
1. Building graph of transactions
2. Inferring $\beta$
3. Inferring $c$
4. Cross-validation process

# Building graph
Throughout the process of model calibration we'll use only part of Ethereum graph. It is placed in file named below

In [4]:
transactions_df = pd.read_csv("./part_data", sep=" ", names=["from", "to", "value", "block"])
transactions_df["value"] = 1
transactions_df = transactions_df.sort_values("block")

In [4]:
transactions_df = pd.read_csv("./Cit-HepPh.txt", names=["from", "to"], sep="\t")
transactions_df["value"] = 1

In [5]:
def create_dataset(dataset, addresses=None):
    if addresses:
        dataset = dataset[dataset["from"].isin(addresses) & dataset["to"].isin(addresses)]
    return dataset.groupby(["from", "to"])["value"].sum().to_frame()

In [6]:
def find_ranks(dataset, alpha):
    edges = dataset["value"].to_dict()
    graph = build_graph(edges)
    nodes = list(graph)
    A = build_matrix(graph, nodes)
    iterations, raw_rank = calculate_spring_rank(A, alpha=alpha)
    
    rank = pd.DataFrame()
    rank["address"] = nodes
    rank["rank"] = raw_rank
    
    return rank.set_index("address")

# Preparing tf

In [ ]:
sess = tf.Session()

In [166]:
device_lib.list_local_devices()

name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5419152033787157668
physical_device_desc: "device: XLA_CPU device"

In [287]:
def get_nessesary_tf_elements(edges, ranks):
    from_indices = tf.transpose(tf.transpose(edges)[0])
    to_indices = tf.transpose(tf.transpose(edges)[1])
    edges_values =  tf.transpose(tf.transpose(edges)[2])

    nodes, reindexed_from_indices = tf.unique(from_indices)
    reindexed_edges = tf.transpose(tf.stack([reindexed_from_indices, to_indices]))

    s = tf.expand_dims(ranks, 1)
    chunk_s = tf.gather_nd(tf.expand_dims(ranks, 1), tf.expand_dims(nodes, 1))

    graph_matrix_shape = [tf.reduce_max(reindexed_from_indices) + 1, tf.shape(ranks)[0]]
    A = tf.sparse_to_dense(sparse_indices=reindexed_edges, sparse_values=edges_values, output_shape=graph_matrix_shape, validate_indices=False)
    A = tf.cast(A, tf.float32)
    
    D = tf.math.sqrt(tf.math.squared_difference(chunk_s, tf.transpose(s)))

    return A, D

# Making ground state energy significance test

In [288]:
nodes_indices = {node: index for index, node in enumerate(np.unique(transactions_df[["from", "to"]].values))}

In [289]:
def randomize_dataset(dataset, probability):
    dataset["random"] = np.random.rand(dataset.shape[0]) > probability
    new_from = dataset["from"] * dataset["random"] + dataset["to"] * (1 - dataset["random"])
    new_to = dataset["from"] * (1 - dataset["random"]) + dataset["to"] * dataset["random"]
    dataset["from"] = new_from
    dataset["to"] = new_to

In [290]:
BATCH_SIZE = 10000
all_edges = tf.placeholder(tf.int32, shape=(None, 3))
edges_dataset = tf.data.Dataset.from_tensor_slices(all_edges).batch(BATCH_SIZE)
edges_iterator = edges_dataset.make_initializable_iterator()
edges_chunk = edges_iterator.get_next()

In [295]:
ranks = tf.placeholder(tf.float32, shape=(None, ))

A, D = get_nessesary_tf_elements(edges_chunk, ranks)
D = (D - 1) * (D - 1)

H_matrix = A * D
H = 0.5 * tf.reduce_sum(H_matrix)

In [292]:
def calculate_H(edges_feed, ranks_feed):
    global H
    total_H = 0
    with tf.device("/device:XLA_CPU:0"):
        sess.run(edges_iterator.initializer, {all_edges: edges_feed})
        while True:
            try:
                total_H += sess.run(H, {ranks: ranks_feed})
            except tf.errors.OutOfRangeError:
                break
        return total_H

Сам тест

In [285]:
prepared_transactions_df = transactions_df.copy()
prepared_transactions_df["from"] = transactions_df["from"].apply(lambda x: nodes_indices[x])
prepared_transactions_df["to"] = transactions_df["to"].apply(lambda x: nodes_indices[x])

In [296]:
calculate_H(state_values, state_ranks)

51593.92010498047

In [278]:
energies = []
for i in tqdm_notebook(range(0, 1)):
    state = create_dataset(prepared_transactions_df)
    state_values = state.reset_index().values
    state_ranks = find_ranks(state, alpha=0).sort_values("address")["rank"].values
    energies += [calculate_H(state_values, state_ranks)]
    randomize_dataset(prepared_transactions_df, 0.5)
    print(energies[0], np.percentile(energies, 20), np.percentile(energies, 50), np.percentile(energies, 70), np.percentile(energies, 90))

Graph contains 421578 edges for 34546 nodes
Estimated size of A is 5.2 MB RAM
Matrix A takes 5.2 MB RAM
Matrix has 3.53e-04 density
01:03:18.032305 Calculating Anj ....
01:03:18.435376 Calculating Ajn ....
01:03:18.638905 Calculating A_o ....
01:03:18.657155 Calculating B ....
01:03:18.672097 Matrix B takes 14.8 MB RAM
01:03:18.672184 Calculating b ....
01:03:18.674923 Solving Bx=b equation using 'bicgstab' iterative method
135981.4189453125 135981.4189453125 135981.4189453125 135981.4189453125 135981.4189453125


In [189]:
print("{0:.0%} percents from ".format(energies[0] / np.percentile(energies, 20)))

29%


# Inferring $\beta$

Except the calculated ranks, the described model has two parameters - temperature $\beta$ and density $c$

To find $\beta$, we'll treat ranks as constant values and apply maximum likelihood procedure described below:

$$L(A|s, \beta) = -\beta H(s) - M \log\sum_{i, j}\exp -\frac{\beta}{2}(s_i - s_j - 1)^2$$

As a result, we'll get $\beta$ parameter that minimizes loss function $L$ in the presence of fixed ranks

In [298]:
nodes_indices = {node: index for index, node in enumerate(np.unique(transactions_df[["from", "to"]].values))}

In [299]:
prepared_transactions_df = transactions_df.copy()
prepared_transactions_df["from"] = transactions_df["from"].apply(lambda x: nodes_indices[x])
prepared_transactions_df["to"] = transactions_df["to"].apply(lambda x: nodes_indices[x])

In [300]:
train_dataset = create_dataset(prepared_transactions_df)
all_edges_feed = train_dataset.reset_index().values

In [301]:
ranks_feed = find_ranks(train_dataset, alpha=0).sort_values("address")["rank"].values

Graph contains 421578 edges for 34546 nodes
Estimated size of A is 5.2 MB RAM
Matrix A takes 5.2 MB RAM
Matrix has 3.53e-04 density
01:05:21.653233 Calculating Anj ....
01:05:21.881943 Calculating Ajn ....
01:05:22.085816 Calculating A_o ....
01:05:22.103507 Calculating B ....
01:05:22.117827 Matrix B takes 14.8 MB RAM
01:05:22.117899 Calculating b ....
01:05:22.120384 Solving Bx=b equation using 'bicgstab' iterative method


In [302]:
BATCH_SIZE = 10000
SHUFFLE_SIZE = 20000
all_edges = tf.placeholder(tf.int32, shape=(None, 3))
edges_dataset = tf.data.Dataset.from_tensor_slices(all_edges).shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE).repeat()
edges_iterator = edges_dataset.make_initializable_iterator()
edges_chunk = edges_iterator.get_next()

In [303]:
# Full-size chunks

ranks = tf.placeholder(tf.float32, shape=(None, ))
beta = tf.Variable(0.0)

A, D = get_nessesary_tf_elements(edges_chunk, ranks)
D = (D - 1) * (D - 1)

log_D = tf.log(tf.reduce_sum(tf.exp(- beta / 2 * D)))

H_matrix = A * D
H = 0.5 * tf.reduce_sum(H_matrix)

M = tf.reduce_sum(A)

In [304]:
# # Batched chunks
# ranks = tf.placeholder(tf.float32, shape=(None, ))
# edges = edges_chunk
# beta = tf.Variable(0.0)

# edges_indices = tf.transpose(tf.transpose(edges)[0:2])
# nodes, reindexed_edges = tf.unique(tf.reshape(edges_indices, shape=(-1, )))
# s = tf.gather_nd(tf.expand_dims(ranks, 1), tf.expand_dims(nodes, 1))
# reindexed_edges = tf.reshape(reindexed_edges, shape=(-1, 2))
# edges_values =  tf.transpose(tf.transpose(edges)[2])
# graph_matrix_shape = [tf.reduce_max(reindexed_edges) + 1] * 2
# A = tf.sparse_to_dense(sparse_indices=reindexed_edges, sparse_values=edges_values, output_shape=graph_matrix_shape, validate_indices=False)
# A = tf.cast(A, tf.float32)

# D = tf.math.squared_difference(s, tf.transpose(s) + 1)
# log_D = tf.log(tf.reduce_sum(tf.exp(- beta / 2 * D)))

# H_matrix = A * D
# H = 0.5 * tf.reduce_sum(H_matrix)

# M = tf.reduce_sum(A)

In [305]:
loss = - beta * H - M * log_D
optimizer = tf.train.AdamOptimizer(0.1).minimize(-loss)

In [306]:
sess.run(tf.global_variables_initializer())
sess.run(edges_iterator.initializer, {all_edges: all_edges_feed})

In [307]:
for i in tqdm_notebook(range(0, 500)):
    feed_dict = {ranks: ranks_feed}
    sess.run(optimizer, feed_dict)
    print("Iteration:", i)
    print("Loss:", sess.run(loss, feed_dict))
    print("Beta:", sess.run(beta, feed_dict))
#     print("Energy:", sess.run(beta * H, feed_dict))
#     print("Sum:", sess.run(M * log_D, feed_dict))

Iteration: 0
Loss: -180908.16
Beta: 0.1
Iteration: 1
Loss: -183836.28
Beta: 0.19991183
Iteration: 2
Loss: -184312.25
Beta: 0.29943252
Iteration: 3
Loss: -184177.34
Beta: 0.39821112
Iteration: 4
Loss: -184178.52
Beta: 0.4965881
Iteration: 5
Loss: -183900.28
Beta: 0.59509087
Iteration: 6
Loss: -183637.25
Beta: 0.69311565
Iteration: 7
Loss: -183495.4
Beta: 0.79018533
Iteration: 8
Loss: -183625.84
Beta: 0.8858615
Iteration: 9
Loss: -183422.98
Beta: 0.9793561
Iteration: 10
Loss: -188263.6
Beta: 1.0662162
Iteration: 11
Loss: -188402.0
Beta: 1.1441854
Iteration: 12
Loss: -187897.98
Beta: 1.2131641
Iteration: 13
Loss: -187066.0
Beta: 1.2758367
Iteration: 14
Loss: -186542.02
Beta: 1.3347199
Iteration: 15
Loss: -186311.97
Beta: 1.3913959
Iteration: 16
Loss: -186065.9
Beta: 1.4453229
Iteration: 17
Loss: -185604.22
Beta: 1.4997649
Iteration: 18
Loss: -185114.67
Beta: 1.5544871
Iteration: 19
Loss: -184689.22
Beta: 1.6103445
Iteration: 20
Loss: -183756.52
Beta: 1.667981
Iteration: 21
Loss: -177442.2

KeyboardInterrupt: 

# Inferring $c$
We'll infer generative model density simply by dividing the sum of predicted edges by the sum of edges that are presented in train dataset.

SpringRank generative model includes formulas to find $E_{ij}$ - number of edges between $i$ and $j$ vertices in both directions, and $P_{ij}$ - proportion of edges that are going from vertex $i$ to vertex $j$. These formulas are going below:

$$P_{ij} = \frac{1}{1 + e^{-2\beta(s_i - s_j)}}, P_{ji} = 1 - P_{ij}$$

$$E_{ij} = c \exp{-\frac{\beta}{2}(s_i - s_j - 1)^2}$$

In [312]:
BATCH_SIZE = 10000
all_edges = tf.placeholder(tf.int32, shape=(None, 3))
edges_dataset = tf.data.Dataset.from_tensor_slices(all_edges).batch(BATCH_SIZE)
edges_iterator = edges_dataset.make_initializable_iterator()
edges_chunk = edges_iterator.get_next()

In [313]:
ranks = tf.placeholder(tf.float32, shape=(None, ))
beta = tf.placeholder(tf.float32)

A, D = get_nessesary_tf_elements(edges_chunk, ranks)
D_squared = (D - 1) * (D - 1)

M = tf.reduce_sum(A)
E = tf.exp(-beta / 2 * D_squared)
P = 1 / (1 + tf.exp(-2 * beta * D))
predicted_M = tf.reduce_sum(E * P)

In [314]:
def infer_density(edges_feed, ranks_feed, beta_feed):
    total_M = 0
    total_predicted_M = 0
    sess.run(edges_iterator.initializer, {all_edges: edges_feed})
    while True:
        try:
            partial_M, partial_predicted_M = sess.run([M, predicted_M], {ranks: ranks_feed, beta: beta_feed})
            total_M += partial_M
            total_predicted_M += partial_predicted_M
        except tf.errors.OutOfRangeError:
            break
    return total_M, total_predicted_M

In [316]:
total_M, total_predicted_M = infer_density(all_edges_feed, ranks_feed, 3.5)

# Cross-validation
Generative model parameters are inferred, so we can try to cross-validate results. High value of metric after cross-validation procedure will be an indicator that we chose $\alpha, \beta, c$ parameters properly, and calcucated ranks are statistically significant

To start a process, we need to calculate edges number between vertices $E_{ij}$ and their directions $p_{ij}$, and to measure the performance of our model. All calculations we should do over the chunked dataset, each chunk will play a role of test part, the rest of dataset will be the train part. The SpringRank paper describes multigraph accuracy as a measure of model performance, formula goes below:
$$\sigma_a = 1 - \frac{1}{2M}\sum_{i,j}|{A_{ij} - (A_{ij} + A_{ji})P_{ij}}|$$
Where $M$ - overall number of edges.

It's important to say that we can deal with multigraph accuracy much faster on sparse graph with skipping empty edges in one and in both directions. We can treat both cases in a way described below:

1. If $A_{ij} \neq 0 $ and $A_{ji} = 0$: $E_{ji}(1 - P_{ji}) + |A_{ji} - E_{ji}P_{ji}|$
2. If $A_{ij} = 0$ and $A_{ji} = 0$: $E_{ij}$

So the accuracy of predictions for such edges can be described by this formula:
$$\sum E_{ij} - \frac{\sum_{(i,j), (j,i) \in G} E_{ij}}{2} - \sum_{(i,j) \in G, (j, i) \in G} E_{ij}$$

In [30]:
def predict_edges(dataset, ranks, beta, c):
    dataset["from_rank"] = ranks.loc[[a for a, _ in dataset.index]]["rank"].tolist()
    dataset["to_rank"] = ranks.loc[[b for _, b in dataset.index]]["rank"].tolist()
    dataset["direction_probability"] = (1 / (1 + np.exp(-2 * beta * (dataset["from_rank"] - dataset["to_rank"]))))
    dataset["number_of_edges"] = c * np.exp(- beta / 2 * (dataset["from_rank"] - dataset["to_rank"] - 1) ** 2)
    return dataset["direction_probability"].tolist(), dataset["number_of_edges"].tolist(), predict_edges_sum(dataset, ranks, beta, c)

In [32]:
def accuracy(dataset, direction, edges, edges_sum):
    accuracy_dataset = dataset.copy()
    accuracy_dataset = accuracy_dataset.merge(accuracy_dataset.reset_index(), left_index=True, right_on=["to", "from"], how="left", suffixes=('', '_reversed'))
    accuracy_dataset["not_paired"] = 0
    accuracy_dataset.loc[np.isnan(accuracy_dataset["value_reversed"]), "not_paired"] = 1
    accuracy_dataset["direction"] = direction
    accuracy_dataset["edges"] = edges
    accuracy_dataset["prediction"] = accuracy_dataset["direction"] * accuracy_dataset["edges"]
    accuracy_dataset["error"] = np.abs(accuracy_dataset["value"] - accuracy_dataset["prediction"])
    accuracy_dataset["non_paired_error"] = accuracy_dataset["not_paired"] * accuracy_dataset["edges"] * (1 - accuracy_dataset["direction"])
    non_active_edges_sum = edges_sum - (accuracy_dataset["not_paired"] * accuracy_dataset["edges"]).sum() - ((1 - accuracy_dataset["not_paired"]) * accuracy_dataset["edges"]).sum() / 2
    return 1 - \
        (accuracy_dataset["error"].sum() + accuracy_dataset["non_paired_error"].sum() + non_active_edges_sum) / \
        (accuracy_dataset["value"].sum() + edges_sum) 

In [33]:
test_df = pd.DataFrame()
test_df["from"] = ["0x1", "0x2", "0x3"]
test_df["to"] = ["0x2", "0x1", "0x4"]
test_df["value"] = [1, 2, 1]
test_df = test_df.set_index(["from", "to"])

In [ ]:
alphas = [0]
# alphas = np.logspace(-2, 2, 5)
betas = [4922.40704411323]
# betas = [288808]
split = ShuffleSplit(n_splits=5)
# split = TimeSeriesSplit(n_splits=5)

train_metrics = {}
test_metrics = {}

train_index, test_index = next(split.split(transactions_df))
# for train_index, test_index in :
train = create_dataset(transactions_df.loc[train_index])
print("Train size: {} samples".format(train.shape[0]))
train_addresses = list([a for a, _ in train.index] + [b for _, b in train.index])
test = create_dataset(transactions_df.loc[test_index], addresses=train_addresses)
print("Test size: {} samples".format(test.shape[0]))
for alpha in alphas:
    ranks = find_ranks(train, alpha=alpha)
    for beta in betas:
        c = infer_density(train, ranks, beta=beta)
        train_directions, train_edges, train_sum = predict_edges(train, ranks, beta=beta, c=c)
        print(beta * c, train_sum)
        test_directions, test_edges, test_sum = predict_edges(test, ranks, beta=beta, c=c)
        print(train_sum, test_sum)
        train_metrics[(alpha, beta, c)] = train_metrics.get((alpha, beta, c), []) + [accuracy(train, train_directions, train_edges, train_sum)]
        test_metrics[(alpha, beta, c)] = test_metrics.get((alpha, beta, c), []) + [accuracy(test, test_directions, test_edges, test_sum)]
        print("Train accuracy: ", train_metrics[(alpha, beta, c)][-1])
        print("Test accuracy: ", test_metrics[(alpha, beta, c)][-1])
pprint(train_metrics)
pprint(test_metrics)

In [155]:
train_metrics_df = pd.DataFrame().from_dict(train_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})
test_metrics_df = pd.DataFrame().from_dict(test_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})

In [156]:
test_metrics_df["alpha_log"] = np.log(test_metrics_df["alpha"])
train_metrics_df["alpha_log"] = np.log(train_metrics_df["alpha"])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


Some cross-validation statistics will be plotted there. Now these cells are empty

In [ ]:
plt.plot(train_metrics_df.groupby("alpha_log")["accuracy"].mean(), label="train")
plt.plot(test_metrics_df.groupby("alpha_log")["accuracy"].mean(), label="test")
plt.legend()

In [ ]:
plt.plot(train_metrics_df.groupby("beta")["accuracy"].max(), label="train")
plt.plot(test_metrics_df.groupby("beta")["accuracy"].max(), label="test")
plt.legend()

In [ ]:
plt.plot(train_metrics_df.groupby("c")["accuracy"].max(), label="train")
plt.plot(test_metrics_df.groupby("c")["accuracy"].max(), label="test")
plt.legend()